In [0]:
# check the contents in FileStore
dbutils.fs.ls("/FileStore/tables")

# Mount S3 bucket to Databricks

In [0]:
# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib

In [0]:
# Specify file type to be csv
file_type = "csv"
# Indicates file has first row as the header
first_row_is_header = "true"
# Indicates file has comma as the delimeter
delimiter = ","
# Read the CSV file to spark dataframe
aws_keys_df = spark.read.format(file_type)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load("/FileStore/tables/authentication_credentials.csv")


In [0]:
# Notes: the secret access key will be encoded using urllib.parse.quote for security purposes. 
# safe="" means that every character will be encoded.

# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
#ACCESS_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Access key ID').collect()[0]['Access key ID']
#SECRET_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")


In [0]:
# mount the S3 bucket by passing in the S3 URL and the desired mount name to dbutils.fs.mount()

# AWS S3 bucket name
AWS_S3_BUCKET = "user-0a5e6ec37a2f-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/pin_data"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

# Create dataframe

In [0]:
# Data about category, description, downloaded, follower_count, image_src
# # File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location = "/mnt/pin_data/topics/0a5e6ec37a2f.pin/partition=0/*.json" 
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
df_pin_raw = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)
# Display Spark dataframe to check its content
display(df_pin_raw)

In [0]:
# Data about country, latitude, longtitude
# # File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location = "/mnt/pin_data/topics/0a5e6ec37a2f.geo/partition=0/*.json" 
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
df_geo_raw = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)
# Display Spark dataframe to check its content
display(df_geo_raw)

In [0]:
# Data about age, date_joined, first_name, ind, last_name
# # File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location = "/mnt/pin_data/topics/0a5e6ec37a2f.user/partition=0/*.json" 
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
df_user_raw = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)
# Display Spark dataframe to check its content
display(df_user_raw)

# Data Cleaning

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Drop duplicates
df_pin = df_pin_raw.dropDuplicates()

# Handle empty entires and entries with error
# Replace them with "Nones"
df_pin = df_pin.replace(['', ' ', 'NULL', 'null'], [None] * 4)
df_pin = df_pin.withColumnRenamed('index', 'ind')
df_pin = df_pin.withColumn("description", when(col("description") == "No description available Story format", None).otherwise(col("description")))
df_pin = df_pin.withColumn("follower_count", when(col("follower_count") == "User Info Error", None).otherwise(col("follower_count")))
df_pin = df_pin.withColumn("image_src", when(col("image_src") == "Image src error.", None).otherwise(col("image_src")))
df_pin = df_pin.withColumn("poster_name", when(col("poster_name") == "User Info Error", None).otherwise(col("poster_name")))
df_pin = df_pin.withColumn("tag_list", when(col("tag_list") == "N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e", None).otherwise(col("tag_list")))
df_pin = df_pin.withColumn("title", when(col("title") == "No Title Data Available", None).otherwise(col("title")))

# Transform follower_count to ensure every entry is a number
# remove non-numeric entries, such that data type is "int"
df_pin = df_pin.withColumn("follower_count", regexp_replace(col("follower_count"), "[^0-9]", ""))
df_pin = df_pin.withColumn("follower_count", col("follower_count").cast(IntegerType()))

# Ensure that each column containing numeric data has a numeric data type
df_pin = df_pin.withColumn("downloaded", col("downloaded").cast(IntegerType()))

# Clean the data in the save_location column to include only the save location path
df_pin = df_pin.withColumn("save_location", regexp_replace(col("save_location"), "Local save in ", ""))

# Rename the index column to ind
df_pin = df_pin.withColumnRenamed("index", "ind")

# Reorder the DataFrame columns
df_pin = df_pin.select(["ind", "unique_id", "title", "description", "follower_count", "poster_name", "tag_list", "is_image_or_video", "image_src", "save_location", "category"])

display(df_pin)



In [0]:
# Drop duplicates
df_geo = df_geo_raw.dropDuplicates()

# Create a new column coordinates that contains an array based on the latitude and longitude columns
df_geo = df_geo.withColumn("coordinates", array(col("latitude"), col("longitude")))

# Drop the latitude and longitude columns from the DataFrame
df_geo = df_geo.drop("latitude", "longitude")

# Convert the timestamp column from a string to a timestamp data type
df_geo = df_geo.withColumn("timestamp", col("timestamp").cast(TimestampType()))

# Reorder the DataFrame columns
df_geo = df_geo.select(["ind", "country", "coordinates", "timestamp"])

display(df_geo)


In [0]:
# Drop duplicates
df_user = df_user_raw.dropDuplicates()

# Create a new column user_name that concatenates the information found in the first_name and last_name columns
df_user = df_user.withColumn("user_name", concat_ws(" ", col("first_name"), col("last_name")))

# Drop the first_name and last_name columns from the DataFrame
df_user = df_user.drop("first_name", "last_name")

# Convert the date_joined column from a string to a timestamp data type
df_user = df_user.withColumn("date_joined", col("date_joined").cast(TimestampType()))

# Reorder the DataFrame columns
df_user = df_user.select(["ind", "user_name", "age", "date_joined"])

display(df_user)
     


# Data Query using Spark on Databricks

In [0]:

from pyspark.sql.window import Window

# Join geo and pin dataframes, on the 'ind' (index) column
df_pin_geo = df_geo.join(df_pin, "ind")

# Group by country and category, then count the frequency
df_category_count = df_pin_geo.groupBy(["country", "category"]).agg(count("*").alias("category_count"))

# Find and extract the most popular category for each country (by sorting within each group)
window = Window.partitionBy("country").orderBy(col("category_count").desc())

df_most_popular = df_category_count.withColumn("rank", rank().over(window)) \
                                   .filter(col("rank") == 1) \
                                   .drop("rank")

#The final dataframe with country, pinterest category, and category count
df_category_final = df_most_popular.select("country", "category", "category_count")

display(df_category_final.select("*"))


In [0]:

df_pin_geo = df_pin.join(df_geo, 'ind', 'inner')

# Convert the timestamp column from string to timestamp type if it's not already
df_pin_geo = df_pin_geo.withColumn("timestamp", col("timestamp").cast("timestamp"))

# Filter the DataFrame for posts between 2018 and 2022
df_5years = df_pin_geo.filter((year("timestamp") >= 2018) & (year("timestamp") <= 2022))

# Create a new column with just the year from the timestamp
df_extracted_year = df_5years.withColumn("post_year", year("timestamp"))

# Group by post_year and category and count the occurrences
df_category_count = df_extracted_year.groupBy("post_year", "category").agg(count("*").alias("category_count"))

# Order the result for better readability
df_result = df_category_count.orderBy("post_year", "category", "category_count")

display(df_result.select("*"))


In [0]:
df_pin_geo = df_pin.join(df_geo, 'ind', 'inner')

# Define window partition by country
window_partition = Window.partitionBy("country").orderBy(col("follower_count").desc())

# Use the window partition specification to add a row number for each user within each country partition
df_ordered = df_pin_geo.withColumn("row_number", row_number().over(window_partition))

# Filter for user with higest follower_count (row_number 1) in each country
df_user_highest_follower_each_country = df_ordered.filter(col("row_number") == 1) \
                                   .select("country", "poster_name", "follower_count")

display(df_user_highest_follower_each_country.select("*"))

In [0]:
# Identify the highest follower count across all country
df_most_followers_all_countries = df_user_highest_follower_each_country.select(max("follower_count")).collect()[0][0]

# Find the country or countries with the user that has the highest global follower count
df_country_with_highest_followers = df_user_highest_follower_each_country.select("*").where(col("follower_count") == df_most_followers_all_countries)

# Display the results
display(df_country_with_highest_followers.select("*"))

In [0]:
df_pin_user = df_pin.join(df_user, 'ind', 'inner')

# Create the age_group categories
df_with_age_categories = df_pin_user.withColumn(
    "age_group",
    when(col("age").between(18, 24), "18-24")
    .when(col("age").between(25, 35), "25-35")
    .when(col("age").between(36, 50), "36-50")
    .otherwise("50+")
)

# Define window partition specification by age_group and ordered by category_count, in descending order
window_partition = Window.partitionBy("age_group").orderBy(col("category_count").desc())

# Filter for the most popular category in each age group
df_highest_category_per_age_group = df_with_age_categories.groupBy("age_group", "category").agg(count("category").alias("category_count")) \
    .withColumn("rank", row_number().over(window_partition)).filter(col("rank") == 1).drop("rank") 

display(df_highest_category_per_age_group)



In [0]:
df_pin_user = df_pin.join(df_user, 'ind', 'inner')

# Create the age_group categories
df_with_age_categories = df_pin_user.withColumn(
    "age_group",
    when(col("age").between(18, 24), "18-24")
    .when(col("age").between(25, 35), "25-35")
    .when(col("age").between(36, 50), "36-50")
    .otherwise("50+")
)

# Copmute the median follower count, for each age group
df_follower_median = df_with_age_categories.groupBy("age_group")\
                                            .agg(percentile_approx("follower_count", 0.5).alias("median_follower_count"))

# Display the result
display(df_follower_median)


In [0]:
# Parse date_joined from string to date type
df_user_query = df_user.withColumn("date_joined", col("date_joined").cast(DateType()))

# Extract the year from date_joined
df_with_year = df_user_query.withColumn("post_year", year(col("date_joined")))

# Filter years from 2015 to 2020
df_6years = df_with_year.filter((col("post_year") >= 2015) & (col("post_year") <= 2020))

# Count users, grouped by post_year
df_users_per_year = df_6years.groupBy("post_year").agg(count("*").alias("number_users_joined"))

display(df_users_per_year)


In [0]:
# Filter users who joined between 2015-2020
df_users_6years = df_user.withColumn("date_joined", col("date_joined").cast("timestamp")) \
                           .withColumn("post_year", year("date_joined")) \
                           .filter((col("post_year") >= 2015) & (col("post_year") <= 2020))

df_users_6years_pin = df_users_6years.join(df_pin, 'ind', 'inner')

# Calculate the median follower count per post year
df_median_follower_count = df_users_6years_pin.groupBy("post_year") \
                                    .agg(expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count"))

display(df_median_follower_count)

In [0]:
# Filter users who joined between 2015-2020
df_users_6years = df_user.withColumn("date_joined", col("date_joined").cast("timestamp")) \
                           .withColumn("post_year", year("date_joined")) \
                           .filter((col("post_year") >= 2015) & (col("post_year") <= 2020))

# Create the age_group categories
df_with_age_categories = df_users_6years.withColumn(
    "age_group",
    when(col("age").between(18, 24), "18-24")
    .when(col("age").between(25, 35), "25-35")
    .when(col("age").between(36, 50), "36-50")
    .otherwise("50+")
)

# merge df_users_age_grouped with df_pin on the user identifier to get follower counts
df_with_age_categories_pin = df_with_age_categories.join(df_pin, 'ind', 'inner')

# Group by age_group and post_year, and calculate the median follower count
df_median_followers = df_with_age_categories_pin.groupBy("age_group", "post_year") \
                               .agg(expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count"))

display(df_median_followers)